In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [3]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [4]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [5]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [6]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.28134798
====> Test set loss: 1.2909, 62.0%
====> Epoch: 150 Average loss: 1.20173033
====> Test set loss: 1.2127, 69.0%
====> Epoch: 225 Average loss: 1.20760253
====> Test set loss: 1.2092, 69.0%
====> Epoch: 300 Average loss: 1.21259918
====> Test set loss: 1.2043, 69.0%
====> Epoch: 375 Average loss: 1.20176897
====> Test set loss: 1.2014, 70.0%
====> Epoch: 450 Average loss: 1.24033634
====> Test set loss: 1.2018, 69.5%
====> Epoch: 525 Average loss: 1.24107657
====> Test set loss: 1.1998, 70.5%
====> Epoch: 600 Average loss: 1.23011834
====> Test set loss: 1.1991, 70.5%
====> Epoch: 675 Average loss: 1.22299410
====> Test set loss: 1.1984, 70.5%
====> Epoch: 750 Average loss: 1.21179613
====> Test set loss: 1.1980, 70.5%
Training state:  False
Elapsed:  46.78293299674988
Complete set accuracy: 72.0%


### Runner

In [8]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(150, 200):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 150
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.22060519
====> Test set loss: 1.1520, 66.5%
====> Epoch: 150 Average loss: 1.20950677
====> Test set loss: 1.1143, 66.5%
====> Epoch: 225 Average loss: 1.16692717
====> Test set loss: 1.1125, 68.0%
====> Epoch: 300 Average loss: 1.20878127
====> Test set loss: 1.1115, 68.0%
====> Epoch: 375 Average loss: 1.19742071
====> Test set loss: 1.1090, 69.0%
====> Epoch: 450 Average loss: 1.18209003
====> Test set loss: 1.1091, 69.0%
====> Epoch: 525 Average loss: 1.18076759
====> Test set loss: 1.1089, 68.5%
====> Epoch: 600 Average loss: 1.17399657
====> Test set loss: 1.1087, 68.5%
====> Epoch: 675 Average loss: 1.13966363
====> Test set loss: 1.1088, 68.5%
====> Epoch: 750 Average loss: 1.20164701
====> Test set loss: 1.1087, 68.5%
Training state:  False
Complete set accuracy: 74.2%
Log accuracy: 72.89999999999999%
---- Done in  49.526108264923096  seconds

---- Running for model name:  B_ad

====> Epoch: 675 Average loss: 1.21063870
====> Test set loss: 1.1445, 73.5%
====> Epoch: 750 Average loss: 1.19216607
====> Test set loss: 1.1439, 73.5%
Training state:  False
Complete set accuracy: 74.6%
Log accuracy: 71.7%
---- Done in  60.38395595550537  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.29354212
====> Test set loss: 1.1702, 73.0%
====> Epoch: 150 Average loss: 1.20758174
====> Test set loss: 1.1259, 71.5%
====> Epoch: 225 Average loss: 1.22319590
====> Test set loss: 1.1147, 72.5%
====> Epoch: 300 Average loss: 1.24702953
====> Test set loss: 1.1075, 72.0%
====> Epoch: 375 Average loss: 1.20182964
====> Test set loss: 1.1045, 73.5%
====> Epoch: 450 Average loss: 1.20160483
====> Test set loss: 1.1041, 73.5%
====> Epoch: 525 Average loss: 1.22531133
====> Test set loss: 1.1036, 73.5%
====> Epoch: 600 Average loss: 1.20085147
====> Test set loss: 1.1032, 73.0%
====> Epoch: 675 Average loss: 1.19036256
====> Test set loss: 1.1028, 7

====> Epoch: 450 Average loss: 1.11351071
====> Test set loss: 1.0981, 70.0%
====> Epoch: 525 Average loss: 1.10827537
====> Test set loss: 1.0978, 70.0%
====> Epoch: 600 Average loss: 1.12972820
====> Test set loss: 1.0978, 70.0%
====> Epoch: 675 Average loss: 1.13274599
====> Test set loss: 1.0979, 70.5%
====> Epoch: 750 Average loss: 1.10511365
====> Test set loss: 1.0975, 71.0%
Training state:  False
Complete set accuracy: 75.2%
Log accuracy: 74.7%
---- Done in  55.963062047958374  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.21303345
====> Test set loss: 1.2099, 66.5%
====> Epoch: 150 Average loss: 1.11985007
====> Test set loss: 1.1761, 69.5%
====> Epoch: 225 Average loss: 1.16281184
====> Test set loss: 1.1767, 69.0%
====> Epoch: 300 Average loss: 1.07431731
====> Test set loss: 1.1851, 68.5%
====> Epoch: 375 Average loss: 1.14198327
====> Test set loss: 1.1832, 68.5%
====> Epoch: 450 Average loss: 1.08966022
====> Test set loss: 1

====> Epoch: 225 Average loss: 1.20012977
====> Test set loss: 1.1394, 72.5%
====> Epoch: 300 Average loss: 1.20306768
====> Test set loss: 1.1476, 71.0%
====> Epoch: 375 Average loss: 1.20088064
====> Test set loss: 1.1503, 70.5%
====> Epoch: 450 Average loss: 1.18949800
====> Test set loss: 1.1491, 70.5%
====> Epoch: 525 Average loss: 1.17452087
====> Test set loss: 1.1485, 70.5%
====> Epoch: 600 Average loss: 1.21237925
====> Test set loss: 1.1499, 70.5%
====> Epoch: 675 Average loss: 1.14133403
====> Test set loss: 1.1503, 70.5%
====> Epoch: 750 Average loss: 1.24718075
====> Test set loss: 1.1508, 70.0%
Training state:  False
Complete set accuracy: 72.2%
Log accuracy: 72.5%
---- Done in  52.51383900642395  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.30627803
====> Test set loss: 1.2778, 64.0%
====> Epoch: 150 Average loss: 1.24112441
====> Test set loss: 1.1827, 69.5%
====> Epoch: 225 Average loss: 1.17156985
====> Test set loss: 1.17

====> Epoch: 75 Average loss: 1.31357245
====> Test set loss: 1.2451, 67.5%
====> Epoch: 150 Average loss: 1.24597260
====> Test set loss: 1.1868, 69.0%
====> Epoch: 225 Average loss: 1.18864207
====> Test set loss: 1.1782, 69.0%
====> Epoch: 300 Average loss: 1.22386854
====> Test set loss: 1.1768, 68.5%
====> Epoch: 375 Average loss: 1.19019839
====> Test set loss: 1.1735, 68.5%
====> Epoch: 450 Average loss: 1.19661381
====> Test set loss: 1.1733, 68.5%
====> Epoch: 525 Average loss: 1.18952634
====> Test set loss: 1.1737, 69.0%
====> Epoch: 600 Average loss: 1.18272874
====> Test set loss: 1.1740, 69.0%
====> Epoch: 675 Average loss: 1.17425637
====> Test set loss: 1.1740, 69.0%
====> Epoch: 750 Average loss: 1.18544467
====> Test set loss: 1.1738, 68.5%
Training state:  False
Complete set accuracy: 71.1%
Log accuracy: 70.89999999999999%
---- Done in  51.84498882293701  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.30933353
====> Test set lo

...

====> Epoch: 525 Average loss: 1.12765184
====> Test set loss: 1.2204, 66.0%
====> Epoch: 600 Average loss: 1.14226627
====> Test set loss: 1.2208, 66.0%
====> Epoch: 675 Average loss: 1.13473779
====> Test set loss: 1.2203, 65.5%
====> Epoch: 750 Average loss: 1.14645704
====> Test set loss: 1.2207, 66.0%
Training state:  False
Complete set accuracy: 76.3%
Log accuracy: 73.0%
---- Done in  66.77147006988525  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.28381001
====> Test set loss: 1.2017, 76.0%
====> Epoch: 150 Average loss: 1.21704147
====> Test set loss: 1.0764, 76.0%
====> Epoch: 225 Average loss: 1.21021326
====> Test set loss: 1.0754, 75.5%
====> Epoch: 300 Average loss: 1.22265807
====> Test set loss: 1.0751, 75.0%
====> Epoch: 375 Average loss: 1.22997444
====> Test set loss: 1.0710, 74.0%
====> Epoch: 450 Average loss: 1.26666285
====> Test set loss: 1.0700, 74.0%
====> Epoch: 525 Average loss: 1.20933438
====> Test set loss: 1.06

====> Epoch: 300 Average loss: 1.21934542
====> Test set loss: 1.1861, 73.0%
====> Epoch: 375 Average loss: 1.22155344
====> Test set loss: 1.1825, 73.0%
====> Epoch: 450 Average loss: 1.18775902
====> Test set loss: 1.1823, 73.0%
====> Epoch: 525 Average loss: 1.18905339
====> Test set loss: 1.1819, 73.0%
====> Epoch: 600 Average loss: 1.22983562
====> Test set loss: 1.1816, 73.5%
====> Epoch: 675 Average loss: 1.23359820
====> Test set loss: 1.1815, 73.5%
====> Epoch: 750 Average loss: 1.20175067
====> Test set loss: 1.1815, 73.5%
Training state:  False
Complete set accuracy: 75.3%
Log accuracy: 72.1%
---- Done in  62.78818106651306  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.28533416
====> Test set loss: 1.2078, 71.5%
====> Epoch: 150 Average loss: 1.21919294
====> Test set loss: 1.1642, 71.5%
====> Epoch: 225 Average loss: 1.21941154
====> Test set loss: 1.1511, 72.0%
====> Epoch: 300 Average loss: 1.17887888
====> Test set loss: 1.1494, 

====> Epoch: 75 Average loss: 1.31682692
====> Test set loss: 1.3406, 55.50000000000001%
====> Epoch: 150 Average loss: 1.23094113
====> Test set loss: 1.2738, 62.0%
====> Epoch: 225 Average loss: 1.22281365
====> Test set loss: 1.2809, 61.0%
====> Epoch: 300 Average loss: 1.21129684
====> Test set loss: 1.2856, 60.0%
====> Epoch: 375 Average loss: 1.24142131
====> Test set loss: 1.2851, 60.0%
====> Epoch: 450 Average loss: 1.21112468
====> Test set loss: 1.2840, 60.0%
====> Epoch: 525 Average loss: 1.23111502
====> Test set loss: 1.2843, 60.0%
====> Epoch: 600 Average loss: 1.26235454
====> Test set loss: 1.2840, 60.0%
====> Epoch: 675 Average loss: 1.19737560
====> Test set loss: 1.2842, 60.0%
====> Epoch: 750 Average loss: 1.25094735
====> Test set loss: 1.2832, 60.5%
Training state:  False
Complete set accuracy: 66.0%
Log accuracy: 69.39999999999999%
---- Done in  64.52165603637695  seconds

---- Running for model name:  D_mild_nadd_lin
====> Epoch: 75 Average loss: 1.19282950
====

====> Epoch: 750 Average loss: 1.21956073
====> Test set loss: 1.0919, 71.0%
Training state:  False
Complete set accuracy: 75.7%
Log accuracy: 72.1%
---- Done in  73.21359300613403  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.23185882
====> Test set loss: 1.1267, 75.0%
====> Epoch: 150 Average loss: 1.22295187
====> Test set loss: 1.0788, 77.0%
====> Epoch: 225 Average loss: 1.18189483
====> Test set loss: 1.0823, 75.5%
====> Epoch: 300 Average loss: 1.16523493
====> Test set loss: 1.0819, 75.5%
====> Epoch: 375 Average loss: 1.15922899
====> Test set loss: 1.0777, 75.5%
====> Epoch: 450 Average loss: 1.14780322
====> Test set loss: 1.0776, 75.5%
====> Epoch: 525 Average loss: 1.16120177
====> Test set loss: 1.0777, 75.5%
====> Epoch: 600 Average loss: 1.14364351
====> Test set loss: 1.0774, 75.5%
====> Epoch: 675 Average loss: 1.16901917
====> Test set loss: 1.0775, 75.5%
====> Epoch: 750 Average loss: 1.14729994
====> Test set loss: 1.

====> Epoch: 525 Average loss: 1.19949359
====> Test set loss: 1.2066, 72.5%
====> Epoch: 600 Average loss: 1.23776347
====> Test set loss: 1.2072, 72.5%
====> Epoch: 675 Average loss: 1.21254862
====> Test set loss: 1.2076, 72.5%
====> Epoch: 750 Average loss: 1.20890805
====> Test set loss: 1.2076, 72.5%
Training state:  False
Complete set accuracy: 75.2%
Log accuracy: 69.3%
---- Done in  65.69502210617065  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.31227820
====> Test set loss: 1.2553, 71.0%
====> Epoch: 150 Average loss: 1.23788291
====> Test set loss: 1.1564, 69.0%
====> Epoch: 225 Average loss: 1.22670212
====> Test set loss: 1.1412, 70.5%
====> Epoch: 300 Average loss: 1.20330668
====> Test set loss: 1.1305, 71.5%
====> Epoch: 375 Average loss: 1.20167903
====> Test set loss: 1.1223, 74.0%
====> Epoch: 450 Average loss: 1.21705026
====> Test set loss: 1.1236, 73.0%
====> Epoch: 525 Average loss: 1.20739387
====> Test set loss: 1.12